In [1]:
!pip install openpyxl
!pip install -U datasets
!pip install -U peft
!pip install -U bitsandbytes
!pip install -U wandb
!pip install -U dataclasses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_excel('/content/sample_data/dataset_comments.xlsx')
test_data = pd.read_excel("/content/sample_data/dataset_comments_35.xlsx")

# Обработка данных и аугментация

In [3]:
data = data.drop(['UserSenderId', 'SubmitDate'], axis=1)
test_data = test_data.drop(['UserSenderId', 'SubmitDate'], axis=1)

In [4]:
from bs4 import BeautifulSoup

# Удалим html теги
def html_to_text(html):
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")
    text = text.replace("\xa0", " ")
    return text.strip()

data.MessageText = data.MessageText.apply(lambda x: html_to_text(x))
test_data.MessageText = test_data.MessageText.apply(lambda x: html_to_text(x))

In [5]:
import re

# Частично избавимся от "шума"
def is_noise(text):
    if len(text) < 3:  # Убираем слишком короткие строки
        return True
    if re.match(r"^[\W\d_]+$", text):  # Только символы и цифры
        return True
    if re.match(r"^[a-zA-Z]+$", text):  # Только латиница
        return True
    return False

data = data[~data.MessageText.apply(is_noise)]

In [ ]:
import torch as t
from transformers import pipeline
from tqdm.notebook import trange, tqdm

paraphrase_model = pipeline(
    "text2text-generation",
    model="cointegrated/rut5-base-paraphraser",
    device=0
)

def paraphrase_text(text, num_return_sequences=3):
    paraphrased = paraphrase_model(
        text,
        max_length=128,
        num_return_sequences=num_return_sequences,
        truncation=True,
        # num_beams=num_return_sequences,
        temperature=0.6,
        do_sample=True,
    )

    return [p["generated_text"] for p in paraphrased]

aug_data = {
    'MessageText': [],
    'labels': []
}

for class_ in data.labels.unique():
    if class_ == 0:
        return_sequences = 10
    elif class_ == 2:
        return_sequences = 3
    elif class_ == 1:
        return_sequences = 2

    subdata = data[data.labels == class_]
    for i in trange(subdata.shape[0]):
        aug_text = paraphrase_text(subdata.iloc[i, 0], return_sequences)

        aug_data['MessageText'] += aug_text
        aug_data['labels'] += [subdata.iloc[i, 1]] * return_sequences

        t.cuda.empty_cache()

aug_data = pd.DataFrame(aug_data)

KeyboardInterrupt: 

In [ ]:
# Рандомно засэмплируем данные и избавимся от дубликатов
aug_data = aug_data.sample(frac=1)

# Объединим данные
data = pd.concat([data, aug_data], axis=0).drop_duplicates()

In [ ]:
# Посмотрим на сбалансированность классов
data.target.value_counts()

,count
target,
1,673
2,656
0,584


# Моделироавние

In [8]:
import numpy as np

from datasets import load_dataset, Dataset

from dataclasses import dataclass
from tqdm.notebook import tqdm

import torch as t
from torch import nn

from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

from peft import LoraConfig, get_peft_model
import wandb

In [10]:
@dataclass
class Config:
    model_name = "cointegrated/rubert-tiny-sentiment-balanced"
    new_model = "seq-cls-ft-system"
    wb_token = 'bab00ed6b8ec6a868aef6917554e2eee8a723676'

config = Config()

In [11]:
# Зайдем в W&B
wandb.login(key=config.wb_token)

run = wandb.init(
    project='Fine-tune Pre-Trained Sequence-Classifier',
    job_type="training"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khhha_pffuuuu (khhha_pffuuuu-saint-petersburg-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [19]:
# Модель
model_name = "cointegrated/rubert-tiny-sentiment-balanced"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Заменяем последний слой и остальные замораживаем
for param in model.parameters():
    param.requires_grad = False

# model.classifier = nn.Linear(312, 3)
model.classifier = nn.Sequential(
    nn.Linear(312, 4096),
    nn.ReLU(),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Linear(4096, 512),
    nn.ReLU(),
    nn.Linear(512, 3)
)

In [15]:
dict_classes = {
    'G': 2,
    'N': 1,
    'B': 0
}

test_data.labels = test_data.labels.apply(lambda x: dict_classes[x])

In [16]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples['MessageText'],
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors="pt"
    )

    return inputs

train_dataset = Dataset.from_pandas(data).map(preprocess_function, batched=True)
test_dataset = Dataset.from_pandas(test_data).map(preprocess_function, batched=True)

Map:   0%|          | 0/1913 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

In [20]:
from sklearn.metrics import recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    predictions = np.argmax(logits, axis=1)

    rec = recall_score(labels, predictions, average="macro")

    return {"eval_recall": rec}

model.to('cuda')

training_args = TrainingArguments(
    output_dir=config.new_model,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=34,
    gradient_accumulation_steps=3,
    dataloader_num_workers=2,
    num_train_epochs=100,
    weight_decay=0.1,
    learning_rate=1e-5,
    optim="adamw_torch",
    lr_scheduler_type='cosine',
    warmup_steps=400,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_recall",
    greater_is_better=False,
    eval_steps=1,
    logging_steps=1,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=training_args,
    compute_metrics=compute_metrics
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Recall
1,1.095400,1.102762,0.464646
2,1.089900,1.098511,0.434343
3,1.087100,1.091758,0.551010
4,1.084300,1.082380,0.697980
5,1.068200,1.070420,0.697980
6,1.066500,1.056125,0.714646
7,1.057500,1.038864,0.684343
8,1.040800,1.017970,0.731313
9,1.016700,0.994138,0.731313
10,0.991400,0.967898,0.731313


TrainOutput(global_step=1000, training_loss=0.6638491753339767, metrics={'train_runtime': 338.3692, 'train_samples_per_second': 565.359, 'train_steps_per_second': 2.955, 'total_flos': 3314830973568000.0, 'train_loss': 0.6638491753339767, 'epoch': 100.0})

In [22]:
# Сохраним модель
model.save_pretrained('/content/model')
tokenizer.save_pretrained('/content/tokenizer')

('/content/tokenizer/tokenizer_config.json',
 '/content/tokenizer/special_tokens_map.json',
 '/content/tokenizer/vocab.txt',
 '/content/tokenizer/added_tokens.json',
 '/content/tokenizer/tokenizer.json')

In [23]:
!zip -r '/content/model.zip' '/content/model'
!zip -r '/content/tokenizer.zip' '/content/tokenizer'

  adding: content/model/ (stored 0%)
  adding: content/model/config.json (deflated 51%)
  adding: content/model/model.safetensors (deflated 8%)
  adding: content/tokenizer/ (stored 0%)
  adding: content/tokenizer/tokenizer.json (deflated 70%)
  adding: content/tokenizer/tokenizer_config.json (deflated 74%)
  adding: content/tokenizer/special_tokens_map.json (deflated 80%)
  adding: content/tokenizer/vocab.txt (deflated 52%)


In [24]:
from google.colab import files

files.download("/content/model.zip")
files.download("/content/tokenizer.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>